In [110]:
import os, sys
currentdir = os.path.dirname(os.path.realpath("Test_Spike-Elimination.ipynb"))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

# MODULES
import datetime
import numpy as np
import scipy.stats as stats
from modules.spike_elimination import GrubbsTestSILLS, OutlierDetection
from modules.essential_functions import Essentials
from modules.spike_elimination import two_sided_test_indices, two_sided_test_outliers

In [111]:
df_test = pd.read_csv("demo_ma05.csv", skiprows=3, skipfooter=1, engine="python")
df_test_01 = df_test.copy()
df_test_02 = df_test.copy()
df_test_03 = df_test.copy()

df_test

,Time [Sec],Li7,Be9,B11,Na23,Al27,Si29,Ti47,Ti49,Mn55,Fe57,Ga69,Ga71,Ge72,Ge73,Sn118
0,0.3446,150.0,50.0,0.0,178384.14,2400.20,63538.65,100.0,50.0,12505.47,10003.51,50.0,0.0,1300.06,0.0,200.0
1,0.6446,150.0,100.0,0.0,180206.34,2100.15,61730.87,50.0,0.0,12955.87,8352.45,100.0,0.0,1350.06,0.0,300.0
2,0.9446,350.0,0.0,50.0,167556.88,2600.23,62233.01,100.0,200.0,12655.61,9202.97,50.0,100.0,1250.06,50.0,150.0
3,1.2447,100.0,0.0,0.0,180307.58,1900.12,59622.09,100.0,150.0,13856.72,9152.94,0.0,0.0,850.03,0.0,300.0
4,1.5447,150.0,0.0,50.0,171805.88,1900.12,62634.73,50.0,100.0,11804.88,9152.94,50.0,0.0,950.03,50.0,150.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,118.8505,200.0,0.0,50.0,180611.32,1900.12,67154.85,100.0,150.0,13206.11,10153.61,50.0,0.0,1500.08,0.0,200.0
396,119.1505,300.0,0.0,100.0,166848.84,2400.20,66652.55,200.0,0.0,12455.43,10303.72,100.0,0.0,1100.04,50.0,200.0
397,119.4506,100.0,0.0,50.0,178485.38,3400.40,61730.87,150.0,350.0,12805.74,10603.94,50.0,50.0,1100.04,50.0,50.0
398,119.7506,300.0,0.0,100.0,170490.60,2800.27,67757.65,100.0,150.0,13406.29,9503.17,100.0,0.0,1350.06,0.0,200.0


In [151]:
list_isotopes = list(df_test.columns)[1:]

alpha = 0.05
threshold = 1000
spikes_grubbs = {}
spikes_whisker = {}

print("Grubbs test (SILLS)")
time_start = datetime.datetime.now()
for isotope in list_isotopes:
    dataset_interval = df_test[isotope]
    dataset_complete = df_test[isotope]
    a, b = GrubbsTestSILLS(
        raw_data=dataset_interval, alpha=alpha, threshold=threshold, start_index=0, 
        dataset_complete=dataset_complete).determine_outlier()
    spikes_grubbs[isotope] = b

    time_end = datetime.datetime.now()
time_delta = (time_end - time_start)*1000
print("Process time:", time_delta.total_seconds(), "ms\n")

print("Whisker test")
time_start = datetime.datetime.now()
for isotope in list_isotopes:
    dataset_interval = df_test[isotope]
    dataset_complete = df_test[isotope]
    a, b = OutlierDetection(
        raw_data=dataset_interval, alpha=alpha, threshold=threshold, isotope=isotope, 
        dataset_complete=dataset_complete).find_outlier()
    spikes_whisker[isotope] = b

    time_end = datetime.datetime.now()
time_delta = (time_end - time_start)*1000
print("Process time:", time_delta.total_seconds(), "ms\n")

Grubbs test (SILLS)
Process time: 848.759 ms

Whisker test
Process time: 11.968 ms



In [152]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

for isotope in list_isotopes:
    print(isotope)
    print("Grubbs test:", spikes_grubbs[isotope], "Whisker test:", spikes_whisker[isotope])
    print("Intersection:", intersection(spikes_grubbs[isotope], spikes_whisker[isotope]))
    print("")

Li7
Grubbs test: [281, 292] Whisker test: [143, 147, 150, 151, 153, 154, 155, 156, 157, 158, 159, 161, 248]
Intersection: []

Be9
Grubbs test: [] Whisker test: []
Intersection: []

B11
Grubbs test: [146, 241] Whisker test: [143, 144, 146, 148, 149, 151, 161, 189, 241, 275]
Intersection: [146, 241]

Na23
Grubbs test: [63, 146] Whisker test: [143, 144, 145, 146, 147, 148, 149, 150, 151, 152]
Intersection: [146]

Al27
Grubbs test: [50, 116, 379] Whisker test: [143, 144, 145, 146, 147, 148, 149, 150, 151]
Intersection: []

Si29
Grubbs test: [231, 266] Whisker test: []
Intersection: []

Ti47
Grubbs test: [152, 290] Whisker test: [152]
Intersection: [152]

Ti49
Grubbs test: [147, 163, 212] Whisker test: [147, 163, 212, 293]
Intersection: [147, 163, 212]

Mn55
Grubbs test: [143, 185, 202, 254, 278] Whisker test: [88, 141, 143, 147, 157, 217, 238, 244, 247, 248, 249, 254, 259, 265, 266, 271, 278, 284, 287, 289, 290, 291, 292, 295, 300, 345]
Intersection: [143, 254, 278]

Fe57
Grubbs test: [216

In [ ]:
def determine_surrounded_values(dataset_interval, index, stepsize=4):
    helper_values = {"POI": 0, "SP": [], "All": []}
    val_poi = dataset_interval[index]
    helper_values["POI"] = val_poi
    helper_values["All"].append(val_poi)

    for step in range(1, stepsize):
        step_before = index - step
        step_after = index + step
        if step_before >= 0:
            helper_values["SP"].append(dataset_interval[step_before])
            helper_values["All"].append(dataset_interval[step_before])
        if step_after < len(dataset_interval):
            helper_values["SP"].append(dataset_interval[step_after])
            helper_values["All"].append(dataset_interval[step_after])

    return helper_values

def calculate_grubbs_critical_value(alpha, size):
    t_dist = stats.t.ppf(1 - alpha/(2*size), size - 2)
    numerator = (size - 1)*np.sqrt(np.square(t_dist))
    denominator = np.sqrt(size)*np.sqrt(size - 2 + np.square(t_dist))
    critical_value = numerator/denominator

    return critical_value

In [149]:
def spike_elimination_grubbs_sills(dataset_interval, dataset_complete, alpha, threshold, start_index):
    helper_data = {}
    smoothed_data = []
    spike_indices = []
    time_a = datetime.datetime.now()
    for index, value in enumerate(dataset_interval):
        if value >= threshold:
            time_0 = datetime.datetime.now()
            
            surrounding_3 = determine_surrounded_values(dataset_interval=dataset_interval, index=index, stepsize=3)
            surrounding_4 = determine_surrounded_values(dataset_interval=dataset_interval, index=index, stepsize=4)
            
            time_1 = datetime.datetime.now()
            time_delta_01 = (time_1 - time_0)*1000000
            if time_delta_01.total_seconds():
                print("Process time 0-1:", time_delta_01.total_seconds(), "us")

            mean_3 = np.mean(surrounding_3["All"])
            std_3 = np.std(surrounding_3["All"], ddof=1)
            mean_4 = np.mean(surrounding_4["All"])
            std_4 = np.std(surrounding_4["All"], ddof=1)
            val_poi_3 = round(abs(value - mean_3)/std_3, 3)
            val_poi_4 = round(abs(value - mean_4)/std_4, 3)
            
            time_2 = datetime.datetime.now()
            time_delta_12 = (time_2 - time_1)*1000000
            if time_delta_12.total_seconds():
                print("Process time 1-2:", time_delta_12.total_seconds(), "us")

            val_crit_3 = round(calculate_grubbs_critical_value(alpha=alpha, size=len(surrounding_3["All"])), 3)
            val_crit_4 = round(calculate_grubbs_critical_value(alpha=alpha, size=len(surrounding_4["All"])), 3)
            
            time_3 = datetime.datetime.now()
            time_delta_23 = (time_3 - time_2)*1000000
            if time_delta_23.total_seconds():
                print("Process time 2-3:", time_delta_23.total_seconds(), "us")

            if val_poi_3 > val_crit_3 and val_poi_4 > val_crit_4:
                val_corr = np.mean(surrounding_3["SP"])
                spike_indices.append(index + start_index)
                helper_data[index + start_index] = val_corr
                
            time_4 = datetime.datetime.now()
            time_delta_34 = (time_4 - time_3)*1000000
            if time_delta_34.total_seconds():
                print("Process time 3-4:", time_delta_34.total_seconds(), "us")
                
    time_b = datetime.datetime.now()
    time_delta_ab = (time_b - time_a)*1000
    print("Process time A-B:", time_delta_ab.total_seconds(), "ms")

    for index, value in enumerate(dataset_complete):
        if index in spike_indices:
            smoothed_data.append(helper_data[index])
        else:
            smoothed_data.append(value)
            
    time_c = datetime.datetime.now()
    time_delta_bc = (time_c - time_b)*1000
    print("Process time B-C:", time_delta_bc.total_seconds(), "ms")

    return smoothed_data, spike_indices

In [150]:
print("Grubbs test (SILLS)")
time_start = datetime.datetime.now()
for isotope in list_isotopes:
    print(isotope)
    dataset_interval = df_test[isotope]
    dataset_complete = df_test[isotope]
    a, b = spike_elimination_grubbs_sills(
        dataset_interval=dataset_interval, dataset_complete=dataset_complete, alpha=alpha, threshold=threshold, start_index=0)

    time_end = datetime.datetime.now()
time_delta = (time_end - time_start)*1000
if time_delta.total_seconds() > 0.0:
    print("Process time:", time_delta.total_seconds(), "ms\n")

Grubbs test (SILLS)
Li7
Process time 2-3: 997.0 us
Process time 1-2: 997.0 us
Process time 0-1: 997.0 us
Process time 2-3: 999.0 us
Process time 3-4: 997.0 us
Process time 1-2: 997.0 us
Process time 1-2: 997.0 us
Process time 2-3: 997.0 us
Process time 2-3: 998.0 us
Process time 1-2: 1025.0 us
Process time 2-3: 969.0 us
Process time 1-2: 998.0 us
Process time 1-2: 1025.0 us
Process time 1-2: 998.0 us
Process time 2-3: 969.0 us
Process time 2-3: 998.0 us
Process time 2-3: 997.0 us
Process time 1-2: 1027.0 us
Process time 2-3: 997.0 us
Process time 0-1: 997.0 us
Process time 0-1: 997.0 us
Process time 2-3: 999.0 us
Process time 2-3: 995.0 us
Process time 2-3: 1000.0 us
Process time 2-3: 997.0 us
Process time 0-1: 996.0 us
Process time 2-3: 997.0 us
Process time 2-3: 1000.0 us
Process time 1-2: 995.0 us
Process time 2-3: 969.0 us
Process time 2-3: 1026.0 us
Process time 2-3: 997.0 us
Process time 0-1: 999.0 us
Process time 2-3: 995.0 us
Process time 2-3: 999.0 us
Process time 2-3: 996.0 u

Process time 2-3: 997.0 us
Process time 2-3: 997.0 us
Process time 2-3: 971.0 us
Process time 0-1: 996.0 us
Process time 2-3: 1028.0 us
Process time 1-2: 994.0 us
Process time 1-2: 998.0 us
Process time 1-2: 997.0 us
Process time 2-3: 970.0 us
Process time 1-2: 998.0 us
Process time 0-1: 997.0 us
Process time 2-3: 997.0 us
Process time 2-3: 998.0 us
Process time 2-3: 997.0 us
Process time 2-3: 997.0 us
Process time 2-3: 1028.0 us
Process time 2-3: 994.0 us
Process time 2-3: 1000.0 us
Process time 2-3: 996.0 us
Process time 2-3: 997.0 us
Process time 2-3: 996.0 us
Process time 0-1: 998.0 us
Process time 2-3: 1003.0 us
Process time 2-3: 996.0 us
Process time 1-2: 993.0 us
Process time 2-3: 997.0 us
Process time 2-3: 997.0 us
Process time 2-3: 998.0 us
Process time 1-2: 1001.0 us
Process time 0-1: 994.0 us
Process time 0-1: 969.0 us
Process time 0-1: 1025.0 us
Process time 1-2: 998.0 us
Process time 2-3: 997.0 us
Process time 2-3: 997.0 us
Process time 2-3: 998.0 us
Process time 2-3: 999.

Process time 2-3: 998.0 us
Process time 2-3: 995.0 us
Process time 2-3: 997.0 us
Process time 2-3: 998.0 us
Process time 1-2: 997.0 us
Process time 2-3: 997.0 us
Process time 1-2: 998.0 us
Process time 2-3: 969.0 us
Process time 0-1: 1026.0 us
Process time 0-1: 997.0 us
Process time 0-1: 997.0 us
Process time 3-4: 969.0 us
Process time 2-3: 1025.0 us
Process time 1-2: 998.0 us
Process time 1-2: 997.0 us
Process time 2-3: 970.0 us
Process time 2-3: 1031.0 us
Process time 2-3: 991.0 us
Process time 2-3: 997.0 us
Process time 2-3: 998.0 us
Process time 2-3: 997.0 us
Process time 1-2: 969.0 us
Process time 2-3: 1030.0 us
Process time 2-3: 993.0 us
Process time 2-3: 1000.0 us
Process time 2-3: 995.0 us
Process time 2-3: 997.0 us
Process time 1-2: 997.0 us
Process time A-B: 80.784 ms
Process time B-C: 0.0 ms
Ti49
Process time 2-3: 997.0 us
Process time 1-2: 1002.0 us
Process time A-B: 1.999 ms
Process time B-C: 0.0 ms
Mn55
Process time 1-2: 997.0 us
Process time 1-2: 998.0 us
Process time 1-

Process time 0-1: 995.0 us
Process time 2-3: 969.0 us
Process time 1-2: 997.0 us
Process time 2-3: 1026.0 us
Process time 1-2: 997.0 us
Process time 2-3: 970.0 us
Process time 3-4: 996.0 us
Process time 2-3: 998.0 us
Process time 0-1: 997.0 us
Process time 2-3: 997.0 us
Process time 2-3: 998.0 us
Process time 0-1: 1026.0 us
Process time 0-1: 969.0 us
Process time 2-3: 1024.0 us
Process time 1-2: 999.0 us
Process time 2-3: 997.0 us
Process time 0-1: 997.0 us
Process time 2-3: 969.0 us
Process time 2-3: 997.0 us
Process time 2-3: 1022.0 us
Process time 2-3: 999.0 us
Process time 2-3: 996.0 us
Process time 0-1: 1001.0 us
Process time 2-3: 997.0 us
Process time 2-3: 997.0 us
Process time 1-2: 998.0 us
Process time 2-3: 997.0 us
Process time 2-3: 997.0 us
Process time 2-3: 998.0 us
Process time 2-3: 997.0 us
Process time 1-2: 997.0 us
Process time 0-1: 998.0 us
Process time 2-3: 997.0 us
Process time 2-3: 1000.0 us
Process time 1-2: 998.0 us
Process time 2-3: 994.0 us
Process time 2-3: 997.